In [1]:
import pandas as pd
df = pd.read_csv('202004-divvy-tripdata.csv')


In [2]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,A847FADBBC638E45,docked_bike,2020-04-26 17:45:14,2020-04-26 18:12:03,Eckhart Park,86,Lincoln Ave & Diversey Pkwy,152.0,41.8964,-87.6610,41.9322,-87.6586,member
1,5405B80E996FF60D,docked_bike,2020-04-17 17:08:54,2020-04-17 17:17:03,Drake Ave & Fullerton Ave,503,Kosciuszko Park,499.0,41.9244,-87.7154,41.9306,-87.7238,member
2,5DD24A79A4E006F4,docked_bike,2020-04-01 17:54:13,2020-04-01 18:08:36,McClurg Ct & Erie St,142,Indiana Ave & Roosevelt Rd,255.0,41.8945,-87.6179,41.8679,-87.6230,member
3,2A59BBDF5CDBA725,docked_bike,2020-04-07 12:50:19,2020-04-07 13:02:31,California Ave & Division St,216,Wood St & Augusta Blvd,657.0,41.9030,-87.6975,41.8992,-87.6722,member
4,27AD306C119C6158,docked_bike,2020-04-18 10:22:59,2020-04-18 11:15:54,Rush St & Hubbard St,125,Sheridan Rd & Lawrence Ave,323.0,41.8902,-87.6262,41.9695,-87.6547,casual


In [3]:
def file_work(df):
    df['started_at'] = pd.to_datetime(df['started_at'])
    df['ended_at'] = pd.to_datetime(df['ended_at'])
    df = df.groupby(['start_station_id','start_station_name','start_lat','start_lng','started_at'])['ride_id'].count()
    df = df.reset_index()
    df = pd.concat({g: x.set_index("started_at").resample("D")['ride_id'].count()
                    for g, x in df.groupby(['start_station_id','start_station_name','start_lat','start_lng'])})
    df = df.reset_index()
    df = df.rename(columns={'level_2':'lat','level_3':'lng','ride_id':'variable_1','level_1':'variable_1_name','level_0':'variable_1_id',"started_at":"date"})
    return df

In [4]:
df = file_work(df)
df.head()

,variable_1_id,variable_1_name,lat,lng,date,variable_1
0,2,Buckingham Fountain,41.8765,-87.6205,2020-04-01,9
1,2,Buckingham Fountain,41.8765,-87.6205,2020-04-02,7
2,2,Buckingham Fountain,41.8765,-87.6205,2020-04-03,11
3,2,Buckingham Fountain,41.8765,-87.6205,2020-04-04,2
4,2,Buckingham Fountain,41.8765,-87.6205,2020-04-05,12


In [5]:
df = df.groupby('date').agg({'variable_1':'sum','lat':'mean','lng':'mean'})
df.reset_index(inplace=True)

In [6]:
df

,date,variable_1,lat,lng
0,2020-04-01,2289,41.903576,-87.655285
1,2020-04-02,2604,41.899783,-87.655043
2,2020-04-03,3522,41.897285,-87.655982
3,2020-04-04,2280,41.895987,-87.656126
4,2020-04-05,2874,41.893927,-87.655876
5,2020-04-06,2421,41.893754,-87.656100
6,2020-04-07,5154,41.892815,-87.655755
7,2020-04-08,2364,41.892847,-87.655808
8,2020-04-09,1972,41.892675,-87.655670
9,2020-04-10,2715,41.892470,-87.655805


In [7]:
import numpy as np
from numpy.polynomial.polynomial import polyval
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import math
from sklearn.metrics import mean_squared_error
from numpy import array
import seaborn as sns
import time
import datetime

# def ai(df, flag):
#     df2 = df.groupby(['lat','lng','variable_1_name'])['variable_1'].count()
#     df2 = df2.reset_index()
#     if flag%2 == 1:
#         X = df2.drop('variable_1_name', axis=1)
#     else:
#         X = df2.drop(['lat','lng','variable_1_name'], axis = 1)
    
#     X_scaled = MinMaxScaler().fit_transform(X)
#     kmeans = KMeans(n_clusters=5, random_state=0).fit(X_scaled)
#     kmeans.labels_
#     label_list = list(kmeans.labels_)
#     df2['labels'] = label_list
#     return df2

# convert an array of values into a dataset matrix for training the LSTM algorithm
def create_dataset(dataset, time_step):
	dataX, dataY = [], []
	for i in range(int(len(dataset))-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return np.array(dataX), np.array(dataY)
    
#this function receives a single column dataframe and use its values to predict a number of values into the future.  
# It receives also the percentage of training data and the number of previous points used to predict one value in the future
#the function returns a new dataframe with the extra predicted values and returns the testing accuracy
def  predict_future_values(df, history_length, number_of_future_values, training_percentage, extrapolation_range):
  print(len(df))
  #length of data should be greater or equal to history_length
  if(len(df)< history_length):
     return 0
  # now we determine the minimum and maximum value of the data
  data_min=df.min()
  data_max=df.max()
  data_range=data_max-data_min
  #now we adjust the minimum value of the scaler given the predicted extrapolation_range
  data_min=data_min-extrapolation_range*data_range
  data_max=data_max+extrapolation_range*data_range
  #first we scale the data using the extrapolated minimum and maximum values
  scaler=MinMaxScaler(feature_range=(0,1))
  scaler.data_min=data_min
  scaler.data_max=data_max
  df1=scaler.fit_transform(np.array(df).reshape(-1,1))
  ##splitting dataset into train and test split
  training_size=int(len(df1)*training_percentage)
  test_size=len(df1)-training_size
  train_data,test_data=df1[0:training_size,:],df1[training_size:len(df1),:1]
  #create the training set
  time_step = history_length
  X_train, y_train = create_dataset(train_data, time_step)
  if time_step >= test_size:
    time_step = test_size -2
  X_test, ytest = create_dataset(test_data, time_step)
  # reshape input to be [samples, time steps, features] as required for LSTM
  X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
  X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
  #create the LSTM Machine learning model and carry out the training
  model=Sequential()
  model.add(LSTM(50,activation='relu', return_sequences=True,input_shape=(history_length,1)))
  model.add(LSTM(50, activation='relu'))
  model.add(Dense(1))
  model.compile(optimizer='adam', loss='mean_squared_error')
  model.fit(X_train,y_train, epochs=200, verbose=1)
  #now we evaluate the accuracy of the model
  ### Lets Do the prediction and check performance metrics
  train_predict=model.predict(X_train)
  test_predict=model.predict(X_test)
  ##Transform back to original form
  train_predict=scaler.inverse_transform(train_predict)
  test_predict=scaler.inverse_transform(test_predict)
  ### Calculate RMSE performance metrics
  training_error=math.sqrt(mean_squared_error(y_train,train_predict))
  testing_error=math.sqrt(mean_squared_error(ytest,test_predict))
  #now that we have the model, we move to predict the required values
  #get the last history values
  data_length=len(test_data)
  x_input=test_data[(data_length-time_step):].reshape(1,-1)
  temp_input=list(x_input)
  temp_input=temp_input[0].tolist()
  # demonstrate prediction for next required future values
  lst_output=[]  #list of output values
  n_steps=time_step
  i=0
  while(i<number_of_future_values):
    if(len(temp_input)>n_steps):
        x_input=np.array(temp_input[1:])
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(x_input)
        print(yhat)
        temp_input.extend(yhat[0].tolist())
        lst_output.extend(yhat.tolist())
        i=i+1
  #we now invert the scale of the output part
  predicted_data=scaler.inverse_transform(lst_output).tolist()
  predicted_data = [x[0] for x in predicted_data]
  return predicted_data

In [8]:
to_ai_df = df['variable_1']
to_ai_df

0     2289
1     2604
2     3522
3     2280
4     2874
5     2421
6     5154
7     2364
8     1972
9     2715
10    4437
11    3655
12    1819
13    1608
14    1417
15    2268
16    1434
17    5077
18    5128
19    3882
20    2625
21    1990
22    2280
23    2283
24    1046
25    6169
26    2588
27    3381
28     644
29    2562
Name: variable_1, dtype: int64

In [9]:
len(to_ai_df)//2

15

In [11]:
print(predict_future_values(to_ai_df,len(to_ai_df)//2,20,0.8,0.1))

30
Epoch 1/200
1/1 [==============================] - 3s 3s/step - loss: 0.2316
Epoch 2/200
1/1 [==============================] - 0s 9ms/step - loss: 0.2196
Epoch 3/200
1/1 [==============================] - 0s 10ms/step - loss: 0.2073
Epoch 4/200
1/1 [==============================] - 0s 10ms/step - loss: 0.1949
Epoch 5/200
1/1 [==============================] - 0s 10ms/step - loss: 0.1826
Epoch 6/200
1/1 [==============================] - 0s 11ms/step - loss: 0.1703
Epoch 7/200
1/1 [==============================] - 0s 11ms/step - loss: 0.1579
Epoch 8/200
1/1 [==============================] - 0s 10ms/step - loss: 0.1455
Epoch 9/200
1/1 [==============================] - 0s 9ms/step - loss: 0.1331
Epoch 10/200
1/1 [==============================] - 0s 10ms/step - loss: 0.1208
Epoch 11/200
1/1 [==============================] - 0s 11ms/step - loss: 0.1087
Epoch 12/200
1/1 [==============================] - 0s 10ms/step - loss: 0.0969
Epoch 13/200
1/1 [==============================] 

In [16]:
new_df=predict_future_values(to_ai_df,len(to_ai_df)//2,20,0.8,0.1)

30
Epoch 1/200
1/1 [==============================] - 7s 7s/step - loss: 0.2755
Epoch 2/200
1/1 [==============================] - 0s 22ms/step - loss: 0.2616
Epoch 3/200
1/1 [==============================] - 0s 16ms/step - loss: 0.2502
Epoch 4/200
1/1 [==============================] - 0s 16ms/step - loss: 0.2407
Epoch 5/200
1/1 [==============================] - 0s 31ms/step - loss: 0.2327
Epoch 6/200
1/1 [==============================] - 9s 9s/step - loss: 0.2252
Epoch 7/200
1/1 [==============================] - 0s 117ms/step - loss: 0.2176
Epoch 8/200
1/1 [==============================] - 0s 77ms/step - loss: 0.2100
Epoch 9/200
1/1 [==============================] - 0s 40ms/step - loss: 0.2026
Epoch 10/200
1/1 [==============================] - 0s 8ms/step - loss: 0.1954
Epoch 11/200
1/1 [==============================] - 0s 10ms/step - loss: 0.1883
Epoch 12/200
1/1 [==============================] - 0s 9ms/step - loss: 0.1811
Epoch 13/200
1/1 [==============================] -

In [13]:
to_ai_df

0     2289
1     2604
2     3522
3     2280
4     2874
5     2421
6     5154
7     2364
8     1972
9     2715
10    4437
11    3655
12    1819
13    1608
14    1417
15    2268
16    1434
17    5077
18    5128
19    3882
20    2625
21    1990
22    2280
23    2283
24    1046
25    6169
26    2588
27    3381
28     644
29    2562
Name: variable_1, dtype: int64

In [14]:
len(to_ai_df)

30

In [19]:
for i in range(len(new_df)):
    to_ai_df[len(to_ai_df)+i+1] = new_df[i]

In [20]:
to_ai_df

0     2289.000000
1     2604.000000
2     3522.000000
3     2280.000000
4     2874.000000
5     2421.000000
6     5154.000000
7     2364.000000
8     1972.000000
9     2715.000000
10    4437.000000
11    3655.000000
12    1819.000000
13    1608.000000
14    1417.000000
15    2268.000000
16    1434.000000
17    5077.000000
18    5128.000000
19    3882.000000
20    2625.000000
21    1990.000000
22    2280.000000
23    2283.000000
24    1046.000000
25    6169.000000
26    2588.000000
27    3381.000000
28     644.000000
29    2562.000000
31    1167.913351
32    1167.913351
34    1172.510350
36    1145.981957
38    1152.875151
40    1128.499123
42    1129.205135
44    1129.846559
46    1130.307725
48    1130.619916
50    1130.579081
52    1130.551665
54    1130.537834
56    1130.534335
58    1130.535982
60    1130.536846
62    1130.537093
64    1130.537093
66    1130.537052
68    1130.537011
70    1130.537011
Name: variable_1, dtype: float64

In [21]:
len(to_ai_df)

51

In [25]:
df= pd.DataFrame(to_ai_df)
df.reset_index(inplace=True)
flag=[]
for i in range(len(to_ai_df)):
    if i<30:
       flag.append(1)
    else:
        flag.append(2)
df['flag'] = flag
df


,index,variable_1,flag
0,0,2289.000000,1
1,1,2604.000000,1
2,2,3522.000000,1
3,3,2280.000000,1
4,4,2874.000000,1
5,5,2421.000000,1
6,6,5154.000000,1
7,7,2364.000000,1
8,8,1972.000000,1
9,9,2715.000000,1
